In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append('..')

import mim.extractors.ab_json as ab_json

In [2]:
specification = {
    "target_label": "label-index+30d-ami+death-30d",
    "json_source": "/home/sapfo/andersb/PycharmProjects/Expect/json_data/pontus_glukos/hbg+lund-train.json.gz",
    "features":{
        "gender",
        "age",
        "bl-Glukos", "bl-TnT", "bl-Krea", "bl-Hb"
    },
}


In [3]:
extractor = ab_json.ABJSONExtractor(specification)

In [4]:
q = extractor.get_data()

['age', 'bl-Glukos', 'bl-Krea', 'bl-TnT']


KeyError: 'Foo'

In [ ]:
q.shape

In [ ]:
q["x"]["numeric"][1]

In [ ]:
q["x"]["categorical"]

In [ ]:
print(ab_json.NUMERICAL_FEATURES)